## Checking APOE

seqlink --fam data/new_trim_ped_famless17_no\:xx.fam --vcf /mnt/mfs/statgen/alzheimers-family/linkage_files/geno/full_sample/vcf/full_sample.vcf.gz --anno MWE/annotation/EFIGA_NIALOAD_chr19.hg38.hg38_multianno.csv --pop data/full_sample_fam_pop.txt -f MERLIN --build hg38 --freq AF -o data/apoechr19test --blueprint data/data/genemap.hg38.chr19APOE.txt --bin 1 --maf-cutoff 0.05 --jobs 1

75 genes
5 Gb


In [ ]:
import numpy as np
import pandas as pd
import pickle

### Read fam

In [ ]:
fam17 = pd.read_csv('../data/new_trim_ped_famless17_no:xx.fam',delim_whitespace=True,header=None,names=['fid','iid','fathid','mothid','sex','ad'])

In [ ]:
fam17.ad[fam17.ad==-9]=0

/tmp/2292762.1.high_mem.q/ipykernel_17476/1308232463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam17.ad[fam17.ad==-9]=0


In [ ]:
fam17.index = list(fam17.iid)

In [ ]:
fam17.ad.value_counts()

2    1900
0    1042
1     957
Name: ad, dtype: int64

## Read haplotypes

In [ ]:
with open('../data/wg20220311/chr19test/tmp/CACHE/chr19test0.pickle', 'rb') as handle:
    apoe = pickle.load(handle)

In [ ]:
apoe.keys()

dict_keys(['WASH5P', 'OR4F17', 'LINC01002', 'PLPP2', 'MIER2', 'THEG', 'C2CD4C', 'SHC2', 'ODF3L2', 'MADCAM1', 'TPGS1', 'CDC34', 'GZMM', 'BSG', 'HCN2', 'POLRMT', 'FGF22', 'RNF126', 'FSTL3', 'PRSS57', 'PALM', 'MISP', 'PTBP1', 'MIR4745', 'PLPPR3', 'MIR3187', 'AZU1', 'PRTN3', 'ELANE', 'CFD', 'MED16', 'RNU6-1@9,RNU6-7@9,RNU6-8@9', 'RNU6-2@9,RNU6-9@9', 'R3HDM4', 'KISS1R', 'ARID3A', 'WDR18', 'GRIN3B', 'TMEM259', 'RNU6-1@10,RNU6-7@10,RNU6-8@10', 'RNU6-2@10,RNU6-9@10', 'CNN2', 'ABCA7', 'ARHGAP45', 'POLR2E', 'GPX4', 'SBNO2', 'STK11', 'CBARP', 'ATP5F1D', 'MIDN', 'CIRBP-AS1', 'CIRBP', 'FAM174C', 'EFNA2', 'PWWP3A', 'NDUFS7', 'GAMT', 'DAZAP1', 'RPS15', 'APC2', 'C19orf25', 'PCSK4', 'REEP6', 'ADAMTSL5', 'PLK5', 'MEX3D', 'MBD3', 'UQCR11', 'TCF3', 'ONECUT3', 'ATP8B3', 'REXO1', 'MIR1909', 'LOC100288123', 'KLF16', 'ABHD17A', 'ADAT3', 'SCAMP4', 'CSNK1G2', 'CSNK1G2-AS1', 'BTBD2', 'MKNK2', 'MOB3A', 'IZUMO4', 'AP3D1', 'DOT1L', 'PLEKHJ1', 'MIR1227', 'SF3A2', 'AMH', 'MIR4321', 'JSRP1', 'OAZ1', 'PEAK3', 'LINGO3',

In [ ]:
apoe['WASH5P']['predata']['1007']

[array(['chr19:62155:A:G', 'chr19:68258:TA:T'], dtype='<U16'),
 array([0.0758, 0.0833]),
 array([['1007', '1007_1', 'A2,1:', '1:'],
        ['1007', '1007_1', '1:', '2:'],
        ['1007', '1007_2', '1:', '2:'],
        ['1007', '1007_2', 'A1,2:', '1:'],
        ['1007', '1007_40', '1:', '?:'],
        ['1007', '1007_40', '?:', '?:'],
        ['1007', '1007_99', 'A1,2:', '1|'],
        ['1007', '1007_99', 'A2,1:', '1|'],
        ['1007', '1007_5', 'A1,2:', '1|'],
        ['1007', '1007_5', 'A2,1:', '1|'],
        ['1007', '1007_3', '1:', '2|'],
        ['1007', '1007_3', '1:', '2|'],
        ['1007', '1007_6', '1:', '2|'],
        ['1007', '1007_6', 'A2,1:', '1|'],
        ['1007', '1007_4', '1:', '2|'],
        ['1007', '1007_4', 'A2,1:', '1|'],
        ['1007', '1007_39', '1:', '2|'],
        ['1007', '1007_39', '1:', '?|']], dtype='<U7')]

## Functions to deal with haplotypes

In [ ]:
def get_allele(s):
    a = s[1] if s[0].isupper() else s[0]
    return 0 if a=='?' else int(a)

In [ ]:
def name_haps(snps):
    name = []
    for i in snps:
        name += [i+'_A0',i+'_A1']
    return name

In [ ]:
def get_fam_hap(haps,name=None):
    new_haps,new_iid = [],[]
    iid = haps[:,1]
    haps = haps[:,2:]
    for i in range(0,haps.shape[0],2):
        new_iid.append(iid[i])
        hap_a01 = []
        for a0,a1 in zip(haps[i],haps[i+1]):
            hap_a01 += [get_allele(a0),get_allele(a1)]
        new_haps.append(hap_a01)
    new_haps = pd.DataFrame(new_haps)
    new_haps.index = new_iid
    if name is not None:
        new_haps.columns = name
    return new_haps

In [ ]:
apoe_predata = apoe['APOE']['predata']

In [ ]:
apoe_freqs = []
for i in apoe_predata.keys():
    if len(apoe_predata[i][0])>0:
        apoe_freqs.append(apoe_predata[i][1])

In [ ]:
apoe_haps = []
for i in apoe_predata.keys():
    if len(apoe_predata[i][0])>0:
        apoe_haps.append(get_fam_hap(apoe_predata[i][2],name_haps(apoe_predata[i][0])))

In [ ]:
apoe_fam_haps = []
for hap in apoe_haps:
    apoe_fam_haps.append(pd.concat([fam17.loc[hap.index],hap],axis=1))

In [ ]:
with open('../data/apoechr19test/apoe_fam_haps.pickle', 'wb') as handle:
        pickle.dump(apoe_fam_haps, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../data/apoechr19test/apoe_freqs.pickle', 'wb') as handle:
        pickle.dump(apoe_freqs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## All genes from haps to peds

In [ ]:
def format_haps_bunch(dhaps,fr,to):
    gene_variants = {}
    gene_haps = {}
    for g in dhaps.keys()[fr:to]:
        haps = dhaps[g]['predata']
        for f in haps.keys():
            if f not in gene_variants.keys():
                gene_variants[f] = {'genes':[],'variants':[],'freqs':[],'uniq':[]}
                gene_haps[f] = get_fam_hap(haps[f][2],name_haps(haps[f][0]))
            else:
                gene_haps[f] = pd.concat([gene_haps[f],get_fam_hap(haps[f][2],name_haps(haps[f][0]))],axis=1)
            gene_variants[f]['genes'] += [g]*len(haps[f][0])
            gene_variants[f]['variants'] += list(haps[f][0])
            gene_variants[f]['freqs'] += list(haps[f][1])
    for i,j in gene_variants.items():
        redup_idx = ~gene_haps[i].columns.duplicated()
        gene_haps[i] = gene_haps[i].iloc[:,redup_idx]
        j['uniq'] = list(redup_idx[range(0,len(redup_idx),2)])
        gene_variants[i] = pd.DataFrame(j)
    return gene_variants,gene_haps

In [ ]:
tmp,tmp1 = format_haps(apoe)

In [ ]:
gene_fam_haps = {}
for k,hap in tmp1.items():
    gene_fam_haps[k] = pd.concat([fam17.loc[hap.index],hap],axis=1)

In [ ]:
with open('../data/apoechr19test/gene_fam_haps.pickle', 'wb') as handle:
        pickle.dump(list(gene_fam_haps.values()), handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../data/apoechr19test/gene_freqs.pickle', 'wb') as handle:
        pickle.dump(tmp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
~gene_fam_haps['1007'].columns.duplicated()[range(1,10,2)]

array([ True,  True,  True,  True,  True])

In [ ]:
gene_fam_haps['1007'].iloc[:,~gene_fam_haps['1007'].columns.duplicated()]

,fid,iid,fathid,mothid,sex,ad,V6-43622586_A0,V6-43622586_A1,V17-43622964_A0,V17-43622964_A1,...,V930-45642984_A0,V930-45642984_A1,V957-45644269_A0,V957-45644269_A1,V965-45644454_A0,V965-45644454_A1,V975-45644979_A0,V975-45644979_A1,V978-45645128_A0,V978-45645128_A1
1007_1,1007,1007_1,0,0,1,0,2,2,2,2,...,1,2,1,1,1,2,2,1,2,1
1007_2,1007,1007_2,0,0,2,0,1,1,1,1,...,1,1,1,2,1,1,2,1,2,1
1007_40,1007,1007_40,0,0,1,0,2,0,1,0,...,1,0,2,0,1,0,1,0,1,0
1007_99,1007,1007_99,1007_1,1007_2,2,2,1,2,1,2,...,1,1,2,1,1,1,1,1,1,1
1007_5,1007,1007_5,1007_1,1007_2,2,2,1,2,1,2,...,1,2,2,1,1,2,1,1,1,2
1007_3,1007,1007_3,1007_1,1007_2,1,2,1,2,1,2,...,1,1,1,1,1,1,2,2,2,2
1007_6,1007,1007_6,1007_1,1007_2,2,2,1,2,1,2,...,1,1,1,1,1,1,2,2,2,2
1007_4,1007,1007_4,1007_1,1007_2,2,0,1,2,1,2,...,1,1,2,1,1,1,1,2,1,2
1007_39,1007,1007_39,1007_40,1007_4,1,1,1,2,1,1,...,1,1,2,2,1,1,1,1,1,1


In [ ]:
tmp1['1010']

In [ ]:
for h in apoe_haps:
    if h.shape[1]<3:
        print(h)

          V39-44907853_A0  V39-44907853_A1
4_146_1                 2                1
4_146_2                 1                1
4_146_99                1                2
4_146_7                 1                1
4_146_6                 1                2
4_146_3                 1                2


In [ ]:
from functools import reduce
apoe_haps_all = reduce(lambda x, y: pd.merge(x, y, how = 'outer',left_index=True,right_index=True), [h.T for h in apoe_haps])

In [ ]:
apoe_haps_all = apoe_haps_all.T

In [ ]:
apoe_haps_all.shape

(2141, 82)

In [ ]:
pd.merge([h.T for h in apoe_haps[:3]],how='outer')

TypeError: merge() missing 1 required positional argument: 'right'

In [ ]:
h.T

,990_1,990_2,990_99,990_12
V5-44906026_A0,1,1,1,1
V5-44906026_A1,2,1,1,2
V25-44907187_A0,1,2,2,2
V25-44907187_A1,2,2,1,2
V49-44908684_A0,2,1,1,1
V49-44908684_A1,1,1,2,1
V58-44909186_A0,1,1,1,1
V58-44909186_A1,2,1,1,2


## chr18 pickle3

In [ ]:
with open('../data/chr18test/tmp/CACHE/chr18test3.pickle', 'rb') as handle:
    chr18pkl3 = pickle.load(handle)

In [ ]:
tmp,tmp1 = format_haps(chr18pkl3)

In [ ]:
gene_fam_haps = {}
for k,hap in tmp1.items():
    gene_fam_haps[k] = pd.concat([fam17.loc[hap.index],hap],axis=1)

In [ ]:
with open('../data/chr18test/chr18pkl3_gene_fam_haps.pickle', 'wb') as handle:
        pickle.dump(gene_fam_haps, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../data/chr18test/chr18pkl3_gene_freqs.pickle', 'wb') as handle:
        pickle.dump(tmp, handle, protocol=pickle.HIGHEST_PROTOCOL)